In [83]:
import sys, importlib
importlib.reload(sys.modules['scrape_reddit'])

from scrape_reddit import search_posts, get_comments

# for debugging
from scrape_reddit import HEADERS
import requests

In [2]:
search_res = search_posts("Coldplay", "stanford")

In [3]:
len(search_res["data"]["children"])

100

In [4]:
search_res["data"]["children"][0]["data"]["name"]

't3_1l13rzc'

In [84]:
comments_res = get_comments("1ihj6ju")

In [85]:
for comment in comments_res['data']['children']:
    children = comment['data']['replies']
    if len(children) > 0:
        print(comment["data"]["id"])
        break

maxhjvm


In [89]:
def process_comments(comments_json: dict[str, object]) -> list[object]:
    for comment in comments_json["data"]["children"]:
        print(comment)
        break
process_comments(comments_res)

{'kind': 't1', 'data': {'subreddit_id': 't5_3b6af', 'approved_at_utc': None, 'author_is_blocked': False, 'comment_type': None, 'awarders': [], 'mod_reason_by': None, 'banned_by': None, 'author_flair_type': 'text', 'total_awards_received': 0, 'subreddit': 'moderatelygranolamoms', 'author_flair_template_id': None, 'likes': None, 'replies': '', 'user_reports': [], 'saved': False, 'id': 'maxfh6u', 'banned_at_utc': None, 'mod_reason_title': None, 'gilded': 0, 'archived': False, 'collapsed_reason_code': None, 'no_follow': True, 'author': 'AutoModerator', 'can_mod_post': False, 'created_utc': 1738680147.0, 'send_replies': False, 'parent_id': 't3_1ihj6ju', 'score': 1, 'author_fullname': 't2_6l4z3', 'approved_by': None, 'mod_note': None, 'all_awardings': [], 'collapsed': False, 'body': "Thanks for your post in r/moderatelygranolamoms! Our goal is to keep this sub a peaceful, respectful and tolerant place. Even if you've been here awhile already please take a minute to [READ THE RULES.](https://

In [41]:
resp = requests.get("https://oauth.reddit.com/comments/1ihj6ju", headers=HEADERS)
comments_json = resp.json()[1]

In [48]:
comments_to_get_more_comments = []
for comment in comments_json['data']['children']:
    children = comment['data']['replies']
    if len(children) > 0:
        comments_to_get_more_comments.append(comment["data"]["id"])
        for child in children["data"]["children"]:
            print(f'{comment["data"]["id"]}: {child["data"]["id"]}')

print(comments_to_get_more_comments)

maxhjvm: maxjoie
maxhjvm: maxo7gn
maxhahm: maxp5lm
maxj7dw: maxq5zo
maxj7dw: maxse2w
maxj7dw: mu7uh5n
maxj7dw: maxqbkd
maxjnd6: mpu01u2
maxjnd6: maxp2bc
maxi6h9: maxkbw1
maxhlsy: mpu0fv3
maxjily: maxjujt
maxjily: maybu2a
maxjily: mswp7zc
maxv304: maxwy79
mlpphlm: myb0l5s
maxjeiz: muxm4df
maxg9mp: mbbqzso
maxhlno: maximgj
['maxhjvm', 'maxhahm', 'maxj7dw', 'maxjnd6', 'maxi6h9', 'maxhlsy', 'maxjily', 'maxv304', 'mlpphlm', 'maxjeiz', 'maxg9mp', 'maxhlno']


In [47]:
children["data"].keys()

dict_keys(['after', 'dist', 'modhash', 'geo_filter', 'children', 'before'])

In [ ]:
for comment in comments_to_get_more_comments:
    resp = requests.get("https://oauth.reddit.com/api/morechildren?api_type=json&link_id=maxhjvm&children=", headers=HEADERS)
    print(resp.json())
    break

{'message': 'Forbidden', 'error': 403}
